# Connect to bio.tools API

In [33]:
# 
import requests

# a GET request
response = requests.get('https://dev.bio.tools/api/tool/?collectionID=West-Life&format=json',
    # could  "Accept": "application/json;charset=UTF-8"
)
assert 200 == response.status_code
btservices = response.json()['list']


## Convert to einfracentral format

In [45]:
import json

# options, helpdesk, userManual, trainingInformation, 

for btservice in btservices:
    eiservice = {
        # required terms 
        'name': btservice['name'],
        'url': btservice['homepage'],
        'order': btservice['download'][0]['url'] if btservice['download'] else btservice['homepage'], 
        "providers": ['West-Life'],

        # Required, but no good match in bio.tools
        'symbol': 'https://about.west-life.eu/network/west-life/templates/westlife/images/west-life.png', 
        "category": "Category-Software", "subcategory": "Subcategory-Tool",
        "places": ["Place-WW"],
        # see below "lifeCycleStatus": "LifeCycleStatus-Production", "trl": "TRL-9",
        "serviceLevelAgreement": "https://auth.west-life.eu/aai/AUP.pdf", # but see below
        'funding' : "EC funding ",

        # terms that map fairly well
        # could register all 'providers': btservice['collectionID'], # nearest approximation
        'version': btservice['version'][0],
        "lastUpdate": btservice['lastUpdate'],
        'description': btservice['description'],
        "languages": ['Language-'+btservice['language']] if btservice['language'] else ["Language-en"],
        'tags': [ t['term'] for t in btservice['topic']],
        "termsOfUse":  [btservice['license']] or ["https://auth.west-life.eu/aai/AUP.pdf"], # but see below

    }
    if btservice['function']:
        for f in btservice['function']:
            if f['note']:
                tagline = f['note']
                eiservice['tagline'] = tagline if len(tagline)<100 else tagline[0:97]+'...'
                break;
    if 'Mature'==btservice['maturity']:
        eiservice["lifeCycleStatus"] = "LifeCycleStatus-Production"
        eiservice["trl"]= "TRL-9"
    elif 'Emerging'==btservice['maturity']:
        eiservice["lifeCycleStatus"] = "LifeCycleStatus-Beta"
        eiservice["trl"]= "TRL-7" # permissible levels are 9,8,7
    else:
        print('maturity', btservice['maturity'])
        
    for d  in btservice['documentation']:
        if 'Citation instructions' == d['type']:
            pass
        elif 'Terms of Use' == d['type']:
            eiservice['termsOfUse'] = d['url']
            eiservice['serviceLevelAgreement'] = d['url']
    for d  in btservice['credit']:
        # unfortunately "max length for 'funding' is 500 chars"
        #if  'Funding agency' == d['typeEntity']:
            # eiservice['funding'] = eiservice['funding']+', '+d['name']
        if  'Primary Contact' in d['typeRole']:
            eiservice['helpdesk'] = d['url']

    # validate it        
    HEADERS = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    response = requests.post('http://beta.einfracentral.eu/api/service/validate', 
        data=json.dumps(eiservice), headers=HEADERS)

    assert 200 == response.status_code, response.json()


'validated'

In [38]:
btservices

[{'collectionID': ['BioExcel', 'West-Life', 'INSTRUCT'],
  'accessibility': [],
  'additionDate': '2016-08-11T13:31:51Z',
  'owner': 'm.e.trellet@uu.nl',
  'cost': None,
  'download': [{'url': 'https://github.com/haddocking/disvis',
    'note': None,
    'version': None,
    'type': 'Source code'}],
  'validated': 0,
  'availability': None,
  'publication': [{'doi': '10.1093/bioinformatics/btv333',
    'version': None,
    'pmid': None,
    'type': 'Primary',
    'pmcid': None,
    'metadata': {'title': 'DisVis: Quantifying and visualizing accessible interaction space of distance-restrained biomolecular complexes',
     'abstract': '© The Author 2015. Summary: We present DisVis, a Python package and command line tool to calculate the reduced accessible interaction space of distance-restrained binary protein complexes, allowing for direct visualization and quantification of the information content of the distance restraints. The approach is general and can also be used as a knowledge-ba